In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler

# ==========================================
# 1. CONFIGURACIÓN Y CARGA DE DATOS
# ==========================================
filename = "Excel.xlsx"

# Carga de datos
if os.path.exists(filename):
    try:
        df = pd.read_excel(filename, engine='openpyxl')
    except:
        df = pd.read_csv("Excel.xlsx - Hoja1.csv")
else:
    df = pd.DataFrame(columns=["Nombres", "A", "B", "C", "D"])

# --- SIMULACIÓN (Si el archivo está vacío/ceros) ---
if len(df) > 0 and df[['A', 'B', 'C', 'D']].sum().sum() == 0:
    print("⚠️ Datos en 0 detectados. Simulando puntajes aleatorios (0-10)...")
    np.random.seed(42)
    for col in ['A', 'B', 'C', 'D']:
        df[col] = np.random.randint(0, 11, len(df))

# ==========================================
# 2. PREPROCESAMIENTO (MACHINE LEARNING)
# ==========================================
features = ['A', 'B', 'C', 'D']

# Normalizamos para detectar talentos relativos
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df[features]), columns=features)

df['Rol_Principal'] = df_scaled.idxmax(axis=1)
df['Puntaje_Normalizado'] = df_scaled.max(axis=1) 

# Ordenamos por los estudiantes más destacados para asignarlos primero
df_sorted = df.sort_values(by='Puntaje_Normalizado', ascending=False).copy()

# ==========================================
# 3. ALGORITMO DE EQUIDAD PURA
# ==========================================

def calcular_poder_grupo(grupo):
    """Calcula la 'fuerza' actual del grupo sumando los talentos asignados"""
    suma = 0
    for integrante in grupo.values():
        if integrante is not None:
            suma += integrante['Puntaje_Normalizado']
    return suma

def crear_grupos_equitativos(df_input):
    pendientes = df_input.to_dict('records')
    num_grupos = len(pendientes) // 4
    
    # Inicializamos grupos vacíos
    grupos = [{k: None for k in features} for _ in range(num_grupos)]
    sin_asignar = []
    
    # --- FASE 1: Asignación por Rol Principal ---
    for estudiante in pendientes:
        rol = estudiante['Rol_Principal']
        
        # Filtramos grupos que necesiten este rol
        grupos_libres = [g for g in grupos if g[rol] is None]
        
        if grupos_libres:
            # EQUIDAD: Elegimos el grupo que tenga MENOR poder acumulado actualmente
            grupos_libres.sort(key=lambda g: calcular_poder_grupo(g))
            mejor_grupo = grupos_libres[0]
            
            mejor_grupo[rol] = estudiante
            estudiante['Rol_Final'] = rol
            estudiante['Nota'] = "⭐ Experto"
        else:
            # Si todos los espacios para ese rol están llenos, pasa a fase de relleno
            sin_asignar.append(estudiante)

    # --- FASE 2: Relleno de Huecos (Multitarea) ---
    for estudiante in sin_asignar:
        puntajes = {k: estudiante[k] for k in features}
        habilidades_ordenadas = sorted(puntajes, key=puntajes.get, reverse=True)
        
        asignado = False
        for habilidad in habilidades_ordenadas:
            grupos_con_hueco = [g for g in grupos if g[habilidad] is None]
            
            if grupos_con_hueco:
                # Volvemos a priorizar al grupo más débil
                grupos_con_hueco.sort(key=lambda g: calcular_poder_grupo(g))
                target_grupo = grupos_con_hueco[0]
                
                target_grupo[habilidad] = estudiante
                estudiante['Rol_Final'] = habilidad
                estudiante['Nota'] = "🔄 Cobertura"
                asignado = True
                break
        
        if not asignado:
            estudiante['Nota'] = "❌ Sin Cupo"

    return grupos

# Ejecutar proceso
grupos_finales = crear_grupos_equitativos(df_sorted)

# ==========================================
# 4. EXPORTACIÓN Y MÉTRICAS
# ==========================================
datos_exportar = []
print("\n" + "="*50)
print("      ASIGNACIÓN 100% EQUITATIVA")
print("="*50)

for i, grupo in enumerate(grupos_finales):
    poder = calcular_poder_grupo(grupo)
    print(f"\n📂 GRUPO {i+1} (Nivel de Poder: {poder:.2f})")
    for rol in features:
        est = grupo.get(rol)
        if est:
            nombre = est['Nombres']
            print(f"   [{rol}] {nombre[:25]:<25} | Nota: {est[rol]:<2} ({est['Nota']})")
            datos_exportar.append({
                'Grupo': i+1,
                'Rol': rol,
                'Nombres': nombre,
                'Puntaje': est[rol],
                'Tipo': est['Nota']
            })

# Guardar resultado
pd.DataFrame(datos_exportar).to_excel("Grupos_Equitativos_Final.xlsx", index=False)

# Validación final de balanceo
df_res = pd.DataFrame(datos_exportar)
promedios = df_res.groupby('Grupo')['Puntaje'].sum()
print("\n" + "-"*30)
print(f"📊 Suma de notas por grupo:\n{promedios}")
print(f"\nDesviación máxima: {promedios.max() - promedios.min()} puntos")

⚠️ Datos en 0 detectados. Simulando puntajes aleatorios (0-10)...

      ASIGNACIÓN 100% EQUITATIVA

📂 GRUPO 1 (Nivel de Poder: 3.08)
   [A] BELLO ORTIZ, CARLOS DAVID | Nota: 10 (⭐ Experto)
   [B] ACERO GARCIA, SAMUEL      | Nota: 7  (⭐ Experto)
   [C] VARGAS HENAO, HAROLD STIV | Nota: 1  (🔄 Cobertura)
   [D] GIL GALLEGO, ALEJANDRO    | Nota: 9  (⭐ Experto)

📂 GRUPO 2 (Nivel de Poder: 3.05)
   [A] JEREZ RAMIREZ, MARTIN     | Nota: 10 (⭐ Experto)
   [B] LOPEZ ROMERO, VALENTINA A | Nota: 9  (⭐ Experto)
   [C] VERGARA SUAREZ, JONATAN D | Nota: 7  (⭐ Experto)
   [D] URREA LARA, DEIVID NICOLA | Nota: 4  (⭐ Experto)

📂 GRUPO 3 (Nivel de Poder: 3.32)
   [A] BELTRAN URBINA, ANDRES CA | Nota: 7  (⭐ Experto)
   [B] CARRILLO PIÑEROS, SANTIAG | Nota: 9  (⭐ Experto)
   [C] CONEJO GOMEZ, ANDREY ESTE | Nota: 9  (⭐ Experto)
   [D] OCAMPO MANRIQUE, JUAN EST | Nota: 7  (⭐ Experto)

📂 GRUPO 4 (Nivel de Poder: 3.22)
   [A] CASTELLANOS ESTUPIÑAN, SE | Nota: 9  (⭐ Experto)
   [B] VALLE MORENO, MARIANA     |